In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Imports

In [4]:
import numpy as np
from ICARUS.propulsion.engine import Engine
from ICARUS.database import DB

#  Load Plane and Engine

In [5]:
engine_dir = "../Data/Engine/Motor_1/"

engine = Engine()
engine.load_data_from_df(engine_dir)
engine.plot_engine_map()

# Vehicle

In [6]:
from ICARUS.vehicle.plane import Airplane
from ICARUS.mission.mission_vehicle import MissionVehicle

In [8]:
plane: Airplane = DB.get_vehicle('final_design')

mission_plane = MissionVehicle(
    plane,
    engine,
    solver= "AVL"
)

Error decoding Plane object final_design! Got error 'dict' object has no attribute 'name'


AttributeError: 'dict' object has no attribute 'name'

# Compute Trajectory

In [7]:
from ICARUS.geometry.polynomial import h_polynomial_factory
from ICARUS.mission.trajectory.trajectory import MissionTrajectory
from ICARUS.mission.trajectory.integrate import RK45_scipy_integrator
import jax.numpy as jnp

In [8]:
operating_floor = 12.5
t0 = 0
x0 =  jnp.array([0., 20.])

coeffs = jnp.array(
    [
        x0[1],
        0.2,
        -1/300
    ]
)
traj_fun_1 = h_polynomial_factory(coeffs)

traj  = MissionTrajectory(
    "SomeTrajectory", 
    traj_fun_1, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)

# Polynomial Trajectory

In [9]:
import jax.numpy as jnp
# velocity is in the same direction as the derivative of the polynomial
g1 = jnp.arctan(
    traj.dy_dx_fd(x0[0])
)
v0 = np.array([np.cos(g1), np.sin(g1)]) * 20.

dt = 0.1
tend = 50
t, xs, vs = RK45_scipy_integrator(t0, tend, dt, x0, v0, traj )

# Plot Trajectory


	Time: 0
	Position: [ 0. 20.]
	Velocity: [20.  0.]
	Gamma = 0.0
	Velocity: 20.0
	AoA: -1.9424447497386603
	alpha: -0.03390205642101689
	Thrust_x = [8.99419326]
	Thrust_y = [-0.00532189]
	Thrust: [8.99419484]
	Drag = 2.194844652012206
	Lift = -22.73633947197516
	Weight = 31.0977


	Time: 5.389578890183418e-07
	Position: [1.07791578e-05 2.00000000e+01]
	Velocity: [ 2.00000012e+01 -9.15367462e-06]
	Gamma = 0.0
	Velocity: 20.000001156015525
	AoA: -1.9424452822808678
	alpha: -0.03390206571563181
	Thrust_x = [8.99419262]
	Thrust_y = [-0.00532189]
	Thrust: [8.99419419]
	Drag = 2.194844905739663
	Lift = -22.736342100331377
	Weight = 31.0977


	Time: 1.0779157780366837e-05
	Position: [2.15583156e-04 2.00000000e+01]
	Velocity: [ 2.00000231e+01 -1.83073492e-04]
	Gamma = 0.0
	Velocity: 20.000023121106498
	AoA: -1.9424554009328074
	alpha: -0.03390224231942402
	Thrust_x = [8.99418034]
	Thrust_y = [-0.00532191]
	Thrust: [8.99418192]
	Drag = 2.194849726738834
	Lift = -22.736392040938178
	Weight = 31.0

C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa =

	Time: 0.020945065773119324
	Position: [ 0.4193707  19.99627137]
	Velocity: [20.04480451 -0.35608445]
	Gamma = 0.5464151336010201
	Velocity: 20.047967081804686
	AoA: 85.23866321708257
	alpha: 2.0341104567376935
	Thrust_x = [8.96175082]
	Thrust_y = [0.31829314]
	Thrust: [8.96740142]
	Drag = 17.243388493075177
	Lift = 110.24805968549316
	Weight = 31.0977


	Time: 0.022607580573676626
	Position: [ 0.45269787 19.99565421]
	Velocity: [20.07596468 -0.61300677]
	Gamma = 0.0
	Velocity: 20.085321384362338
	AoA: -1.9815114053605396
	alpha: -0.03458389818936143
	Thrust_x = [8.94654168]
	Thrust_y = [-0.00540016]
	Thrust: [8.94654331]
	Drag = 2.2136113151676993
	Lift = -22.93074285440513
	Weight = 31.0977


	Time: 0.024685724074373257
	Position: [ 0.4942246  19.99598848]
	Velocity: [20.02638454 -0.2007561 ]
	Gamma = 0.0
	Velocity: 20.02739076114387
	AoA: -1.9550382089341234
	alpha: -0.03412185374819327
	Thrust_x = [8.97889209]
	Thrust_y = [-0.00534728]
	Thrust: [8.97889368]
	Drag = 2.20086061529965

C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa =

	Time: 0.018482353197306144
	Position: [ 0.37001281 19.99709739]
	Velocity: [20.03954937 -0.31417827]
	Gamma = 0.5464151336010201
	Velocity: 20.042012049387
	AoA: -85.59805075132186
	alpha: -0.9475526852987525
	Thrust_x = [8.96950049]
	Thrust_y = [-0.14835035]
	Thrust: [8.97072722]
	Drag = 2.204075329023403
	Lift = -22.831959818449487
	Weight = 31.0977


	Time: 0.018482353197306144
	Position: [ 0.37001284 19.99709748]
	Velocity: [20.03960529 -0.31422037]
	Gamma = 0.5464151336010201
	Velocity: 20.042068620316414
	AoA: -85.52965763201198
	alpha: -0.9463590001810857
	Thrust_x = [8.96947199]
	Thrust_y = [-0.14816296]
	Thrust: [8.97069562]
	Drag = 2.2040877715631564
	Lift = -22.832088710399752
	Weight = 31.0977


	Time: 0.021393248224541327
	Position: [ 0.42834603 19.99618282]
	Velocity: [20.04601772 -0.36389652]
	Gamma = 0.0
	Velocity: 20.04932036820047
	AoA: -1.9650853545584321
	alpha: -0.0342972095197648
	Thrust_x = [8.96664404]
	Thrust_y = [-0.00536743]
	Thrust: [8.96664565]
	Drag = 2.2

...

	Time: 0.08128533924020821
	Position: [ 1.63242404 19.94659972]
	Velocity: [20.13972633 -1.10755165]
	Gamma = 0.5464151336010201
	Velocity: 20.170157347513896
	AoA: 91.92893879512802
	alpha: 2.1508777934403525
	Thrust_x = [8.8929275]
	Thrust_y = [0.33399651]
	Thrust: [8.89919734]
	Drag = 17.45422235039298
	Lift = 111.59605597373026
	Weight = 31.0977


	Time: 0.08155472485360626
	Position: [ 1.63784884 19.94630598]
	Velocity: [20.13890904 -1.10069994]
	Gamma = 0.0
	Velocity: 20.16896619920038
	AoA: -2.019321725718159
	alpha: -0.035243812770835735
	Thrust_x = [8.89986016]
	Thrust_y = [-0.00547449]
	Thrust: [8.89986185]
	Drag = 2.2320867625938776
	Lift = -23.122129540562216
	Weight = 31.0977


	Time: 0.08290165292059651
	Position: [ 1.66497175 19.94484647]
	Velocity: [20.18404287 -1.47396535]
	Gamma = 0.5464151336010201
	Velocity: 20.237790401954154
	AoA: -88.14477573986018
	alpha: -0.9920014214915609
	Thrust_x = [8.86015089]
	Thrust_y = [-0.15341724]
	Thrust: [8.86147903]
	Drag = 2.24734

C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa =

	Time: 0.0872094032299458
	Position: [ 1.75147033 19.94219424]
	Velocity: [20.4105199  -3.31870351]
	Gamma = 0.5464151336010201
	Velocity: 20.678566577265816
	AoA: -90.87051795185457
	alpha: -1.039574597651456
	Thrust_x = [8.61494461]
	Thrust_y = [-0.15632671]
	Thrust: [8.61636285]
	Drag = 2.346306029142159
	Lift = -24.305323994028853
	Weight = 31.0977


	Time: 0.08768052561526495
	Position: [ 1.76088672 19.94226118]
	Velocity: [20.39305905 -3.17502707]
	Gamma = 0.5464151336010201
	Velocity: 20.638741589005132
	AoA: 87.62593265169932
	alpha: 2.075776168503979
	Thrust_x = [8.63278501]
	Thrust_y = [0.31289519]
	Thrust: [8.63845359]
	Drag = 18.274620152619207
	Lift = 116.84138614198618
	Weight = 31.0977


	Time: 0.08768052561526495
	Position: [ 1.76127462 19.93908936]
	Velocity: [20.15203325 -1.20497331]
	Gamma = 0.5464151336010201
	Velocity: 20.188026268924936
	AoA: -94.05570364653198
	alpha: -1.0951665753109998
	Thrust_x = [8.88760579]
	Thrust_y = [-0.16990072]
	Thrust: [8.8892296]
	Dra

C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]
C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\mission\trajectory\trajectory.py:146: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  aoa = fsolve(f, np.deg2rad(aoa_prev), xtol=1e-07)[0]


KeyboardInterrupt: 

In [ ]:
traj.plot_history()

# Optimization